In [57]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
from yahooquery import Ticker

In [58]:
year = 2022
df = pd.read_csv(f'Data/sp500_wrds_{year}_fundamental.csv')
price_df = pd.read_csv(f'Data/sp500_wrds_{year}_price.csv') # include 12 months before data
# df.head()

In [59]:
len(df.columns)

65

In [60]:
# initial_columns = ['permno', 'adate', 'qdate', 'public_date', 'bm', 'evm', 'pe_op_basic',
#        'pe_op_dil', 'pe_exi', 'ps', 'pcf', 'dpr', 'npm', 'opmad', 'gpm',
#        'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
#        'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'GProf',
#        'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
#        'int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
#        'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
#        'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
#        'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'inv_turn',
#        'at_turn', 'rect_turn', 'pay_turn', 'sale_equity', 'sale_nwc',
#        'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
#        'divyield', 'TICKER']
# # some are dropped due to high missing values or redundancy after this block

# df = df[initial_columns]

In [61]:
cols_to_drop = ['permno','adate', 'qdate','PEG_trailing','inv_turn', 'sale_nwc','profit_lct','ocf_lct','curr_debt', 'pretret_noa', 'intcov', 'intcov_ratio', 'rect_act','invt_act']

In [62]:
df = df.drop(columns=cols_to_drop)
df = df.rename(columns={'TICKER': 'ticker', 'public_date': 'date'})
df = df.dropna(subset=['ticker'])

In [63]:
# df.isna().sum()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# df.set_index('date', inplace=True)

In [64]:
#remoce % sign from divyield and convert to float
df["divyield"] = df["divyield"].str.rstrip('%')
df["divyield"] = df["divyield"].fillna(0)
df["divyield"] = df["divyield"].astype('float')

In [65]:
df = df.sort_values(['ticker', 'date'])
df = df.groupby('ticker').apply(lambda x: x.ffill().bfill())
df = df.reset_index(drop=True)

In [66]:
df.head()

,date,bm,evm,pe_op_basic,pe_op_dil,pe_exi,ps,pcf,dpr,npm,...,rect_turn,pay_turn,sale_equity,rd_sale,adv_sale,staff_sale,accrual,ptb,divyield,ticker
0,2022-01-31,0.115,29.262,33.251,33.652,35.360,6.658,28.333,0.195,0.191,...,5.719,6.955,1.173,0.070,0.01,0.0,-0.027,7.661,0.557,A
1,2022-02-28,0.115,29.262,31.112,31.488,33.086,6.197,26.367,0.195,0.191,...,5.719,6.955,1.173,0.070,0.01,0.0,-0.027,7.130,0.595,A
2,2022-03-31,0.123,26.424,31.810,32.119,33.586,6.162,26.441,0.199,0.187,...,5.636,6.461,1.270,0.071,0.01,0.0,-0.029,7.705,0.000,A
3,2022-04-30,0.123,26.424,28.671,28.949,30.272,5.554,23.831,0.199,0.187,...,5.636,6.461,1.270,0.071,0.01,0.0,-0.029,6.945,0.704,A
4,2022-05-31,0.123,26.424,30.663,30.961,32.376,5.940,25.488,0.199,0.187,...,5.636,6.461,1.270,0.071,0.01,0.0,-0.029,7.428,0.659,A


2022 specific

In [67]:
# renam fb ticker to meta (2022 change)
df['ticker'] = df['ticker'].replace({'FB': 'META'})

In [68]:
print(len(df["ticker"].unique()))

469


In [69]:
price_df.columns

Index(['tic', 'datadate', 'gvkey', 'ggroup', 'gind', 'gsector', 'cshtrm',
       'prccm'],
      dtype='object')

In [70]:
# create a dictionary to map ticker to gind
gind_mapping = price_df.set_index('tic')['gind'].to_dict()
gsec_mapping = price_df.set_index('tic')['gsector'].to_dict()
df['gind'] = df['ticker'].map(gind_mapping)
df['gsector'] = df['ticker'].map(gsec_mapping)


price_df = price_df[['tic', 'datadate', 'cshtrm', 'prccm']]
price_df = price_df.rename(columns={'tic': 'ticker', 'datadate': 'date', 'cshtrm': 'volume', 'prccm': 'price'}) # rename columns
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d')
# price_df.set_index('date', inplace=True)

In [71]:
group = price_df.groupby('ticker')

# 1. Simple Moving Averages
price_df['SMA_3']  = group['price'].transform(lambda x: x.rolling(3).mean())
price_df['SMA_6']  = group['price'].transform(lambda x: x.rolling(6).mean())
# price_df['SMA_12'] = group['price'].transform(lambda x: x.rolling(12).mean())

# 2. Price Returns / Momentum
price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))
# price_df['ret_12m'] = group['price'].transform(lambda x: x.pct_change(12))

# 3. Volatility (Rolling Std of returns)
price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())
# price_df['vol_12m'] = group['ret_1m'].transform(lambda x: x.rolling(12).std())

# 4. Exponential Moving Average
price_df['EMA_6']  = group['price'].transform(lambda x: x.ewm(span=6, adjust=False).mean())
price_df['EMA_12'] = group['price'].transform(lambda x: x.ewm(span=12, adjust=False).mean())

# 5. RSI (approx price-only RSI)
def rsi(series, period=6):
    delta = series.diff()
    gain = delta.clip(lower=0).rolling(period).mean()
    loss = -delta.clip(upper=0).rolling(period).mean()
    rs = gain / loss
    return 100 - (100/(1 + rs))

price_df['RSI'] = group['price'].transform(rsi)

# 6. MACD
exp1 = group['price'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
exp2 = group['price'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
price_df['MACD_ratio'] = (exp1 - exp2) / price_df['price']


# 7. Volume moving average
price_df['vol_SMA_3'] = group['volume'].transform(lambda x: x.rolling(3).mean())
price_df['vol_SMA_6'] = group['volume'].transform(lambda x: x.rolling(6).mean())

# 8. Volume Rate of Change
price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# 9. OBV (On-Balance Volume)
def obv(df):
    sign = df['price'].diff().apply(lambda x: 1 if x>0 else -1 if x<0 else 0)
    return (sign * df['volume']).cumsum()

price_df['OBV'] = group.apply(obv).reset_index(level=0, drop=True)


In [72]:
price_df.columns

Index(['ticker', 'date', 'volume', 'price', 'SMA_3', 'SMA_6', 'ret_1m',
       'ret_3m', 'ret_6m', 'vol_3m', 'vol_6m', 'EMA_6', 'EMA_12', 'RSI',
       'MACD_ratio', 'vol_SMA_3', 'vol_SMA_6', 'vol_ROC', 'OBV'],
      dtype='object')

In [73]:
nans_col = df.isna().sum()
missing_columns = nans_col[nans_col > 0]

for col in missing_columns.index:
    if col not in ['gind', 'gsector']:
        # 1. Fill using industry mean
        df[col] = df[col].fillna(df.groupby('gind')[col].transform('mean'))
        # 2. Fill remaining using sector mean
        df[col] = df[col].fillna(df.groupby('gsector')[col].transform('mean'))
        

In [74]:
# df.isna().sum()

In [75]:
df.columns

Index(['date', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'ps', 'pcf',
       'dpr', 'npm', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce',
       'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity', 'GProf',
       'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
       'int_debt', 'int_totdebt', 'cash_lt', 'debt_at', 'debt_ebitda',
       'short_debt', 'lt_debt', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be',
       'debt_assets', 'debt_capital', 'de_ratio', 'at_turn', 'rect_turn',
       'pay_turn', 'sale_equity', 'rd_sale', 'adv_sale', 'staff_sale',
       'accrual', 'ptb', 'divyield', 'ticker', 'gind', 'gsector'],
      dtype='object')

In [76]:
# df.isna().sum()

In [77]:
# all other missing values imputed using KNN imputer
from sklearn.impute import KNNImputer

num_cols = df.select_dtypes(include='number').columns
imputer = KNNImputer(n_neighbors=3)
df[num_cols] = imputer.fit_transform(df[num_cols])

# still check for any missing values then simple fillna with column mean for numerical columns
df.drop(['gind', 'gsector'], axis=1, inplace=True)
num_cols = df.select_dtypes(include='number').columns
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [78]:
price_tickers = price_df['ticker'].unique()
fundamentals_tickers = df['ticker'].unique()
common_tickers = set(price_tickers).intersection(set(fundamentals_tickers))
print(f"Number of unique tickers in fundamentals data: {len(fundamentals_tickers)}")
print(f"Number of unique tickers in price data: {len(price_tickers)}")
print(f"Number of common tickers: {len(common_tickers)}")

Number of unique tickers in fundamentals data: 469
Number of unique tickers in price data: 510
Number of common tickers: 443


In [79]:
# Need to manually check which tickers are missing in either dataset
missing_tickers = set(fundamentals_tickers) - common_tickers
print(f"Tickers in fundamentals data but missing in price data: {missing_tickers}")

Tickers in fundamentals data but missing in price data: {'PKI', 'BH', 'CMT', 'MLSS', 'WABC', 'FRC', 'FBHS', 'TCX', 'CDAY', 'AMR', 'COG', 'COOP', 'LEE', 'DISCA', 'PAR', 'APLD', 'FLT', 'ANTM', 'WW', 'ITI', 'NLOK', 'PARA', 'BLL', 'ATI', 'SIVB', 'ABC'}


In [80]:
df = df.set_index(['date', 'ticker'])
price_df = price_df.set_index(['date', 'ticker'])
df_merged = df.join(price_df, how='inner')
df_merged.reset_index(inplace=True)

In [81]:
df_merged.shape, len(df_merged['ticker'].unique())

((5171, 68), 443)

In [82]:
# df_merged.isna().sum()

In [83]:
# show tickers with a single missing values
missing_tickers = df_merged[df_merged.isna().any(axis=1)]['ticker'].unique()
missing_tickers # investigate these tickers

array(['CEG'], dtype=object)

In [84]:
# droping these tickers
df_merged = df_merged[~df_merged['ticker'].isin(missing_tickers)]
df_merged.isna().sum().sum()

0

In [85]:
# if above is not 0, see why

In [87]:
df_merged[cols].corr().shape

(66, 66)

In [89]:
# save csv to be used in modeling
df_merged.to_csv(f'Data/sp500_cleaned_{year}.csv', index=False)

In [91]:
df_merged.shape

(5163, 68)

In [102]:
# get spy montlhy last day from yfinance
import yfinance as yf
import pandas as pd

spy = yf.download("SPY", start="2019-01-01", interval="1d")
spy_monthly = spy.resample('M').last()





[*********************100%***********************]  1 of 1 completed


In [104]:
df_spy = spy_monthly["Close"].reset_index()

In [107]:
df_spy.to_csv('Data/SPY.csv', index=False)

In [108]:
df_spy.head()

Ticker,Date,SPY
0,2019-01-31,243.474564
1,2019-02-28,251.366882
2,2019-03-31,255.916763
3,2019-04-30,266.371613
4,2019-05-31,249.384796
